**Before you start, make sure you use the T4 GPU processor (Runtime->Change Runtime-> T4GPU)**

In [ ]:
#Import drive
from google.colab import drive
#Mount Google Drive
ROOT="/content/drive"
drive.mount(ROOT, force_remount=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms


cudnn.benchmark = True

Send the code to the GPU

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

In [ ]:
%cd /content/drive/MyDrive/dlss24/

In [ ]:
%pwd

# Transformers for classification

In this HW you will do the following steps:


1.   *1 Pts*: Train a transformer model
2.   *1 Pts*: Prediction
3.   *1 Pts*: Metrics
4.   *3 Pts*: Model comparison and discussion



**Before you start, make sure you use the T4 GPU processor (Runtime->Change Runtime-> T4GPU)**


In [ ]:
import pandas as pd
import numpy as np
df_source_corpus=pd.read_csv('/content/drive/MyDrive/dlss24/source_corpus.csv')
df_source_corpus = df_source_corpus.dropna(subset=['text'])

## *1 Pt*: Train Model


> Complete the classification pipeline


1.   Split the data correctly for batches of 8 (line 37)
2.   Add right code in line 48, 50,51

In [ ]:
import torch
from transformers import DistilBertTokenizer, TextClassificationPipeline, DistilBertForSequenceClassification, DistilBertTokenizerFast
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd
from tqdm import tqdm

# Load the dataset
df_sample = df_source_corpus[0:1000]
df_sample = df_sample.assign(topic_id=(df_sample['topic_8']).astype('category').cat.codes)

# Determine the number of unique classes
num_classes = len(set(df_sample['topic_8'].tolist()))
print(f"Number of classes: {num_classes}")

model_name = 'distilbert-base-uncased'
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

# Prepare inputs and labels
inputs = tokenizer(df_sample['text'].tolist(), return_tensors="pt", padding=True, truncation=True)
labels = torch.tensor(df_sample['topic_id'].tolist()).long()

optimizer = torch.optim.Adam([
    {'params': model.distilbert.parameters(), 'lr': 1e-5},
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_sample['text'].tolist(), df_sample['topic_id'].tolist(), test_size=.2)

# Generate batches
X_train, X_test, y_train, y_test = np.array(X_train[:344]), np.array(X_test[:80]), np.array(y_train[:344]), np.array(y_test[:80])

# TODO:  BATCHES OF 8
X_train, X_test, y_train, y_test = ...


X_train, X_test = X_train.tolist(), X_test.tolist()

# Train the model
model.to('cuda:0')
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for text, labels in tqdm(zip(X_train, y_train), total=len(X_train)):
      #TODOD
        model_inputs = ...
        model_inputs = {k: v.to('cuda:0') for k, v in model_inputs.items()}
      #TODO
        labels = ...
        output = ...
        loss, logits = output[:2]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

torch.save(model, 'topic_classifier.pt')
topic_classifier = model

distil_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', max_length=512, truncation=True)
pipeline = TextClassificationPipeline(model=topic_classifier, tokenizer=distil_tokenizer, return_all_scores=True, device=0)







## *1 Pt*: Prediction

> Do predictions on Test dataset

In [ ]:
# Tokenize and predict on the test set
y_true = []
y_pred = []

for text, true_label in zip(X_test, y_test):
    model_inputs = distil_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    model_inputs = {k: v.to('cuda:0') for k, v in model_inputs.items()}

    with torch.no_grad():
       #TODO:
        output = ...
        logits = ...
        predictions = ...

    y_true.extend(true_label)
    y_pred.extend(predictions)

## *1 Pt*: Metrics

> Create the metrics

In [ ]:
category_mapping = {
    'economy': 0,
    'politics': 1,
    'society': 2,
    'freedom and democracy': 3,
    'external relations': 4,
    'fabric of society': 5,
    'welfare and quality of life': 6,
    'no topic': 7,
}




# Ensure y_true and y_pred are in the correct shape
 #TODO:
y_true = ...
y_pred = ..



# Generate classification report
class_report = ...
print("Classification Report:")
print(class_report)


## *3 Pts*: Model comparison and discussion



> Train other transformer model on the same dataset and compare the metrics. Discuss their performance

